In [1]:
import mysql.connector
import json
from datetime import datetime

#function to create CVE tables for each year
def create_database_schema():
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="passwordhere",
            database="CVES4"
        )
        cursor = conn.cursor()

        #schema for CVE records 2020<
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS CVE_2020_before (
                CVE_ID VARCHAR(255) PRIMARY KEY,
                Description TEXT,
                Published_Date DATE,
                Last_Modified_Date DATE,
                CVSS VARCHAR(50),
                CWE VARCHAR(50)             
            )
        """)

        #schema for CVE records 2021-2024
        for year in range(2021, 2025):
            table_name = f"CVE_{year}"
            cursor.execute(f"""
                CREATE TABLE IF NOT EXISTS {table_name} (
                    CVE_ID VARCHAR(255) PRIMARY KEY,
                    Description TEXT,
                    Published_Date DATE,
                    Last_Modified_Date DATE,
                    CVSS VARCHAR(50),
                    CWE VARCHAR(50)
                    )
            """)

        #schema for websites affected
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Affected_Products (
                CVE_ID VARCHAR(255),
                Product_Name VARCHAR(255),
                Vendor VARCHAR(255),
                Version VARCHAR(100),
                FOREIGN KEY (CVE_ID) REFERENCES CVE_2020_before(CVE_ID)
            )
        """)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Vendor (
                Reference_ID INT AUTO_INCREMENT PRIMARY KEY,
                URL VARCHAR(1000),
                Description TEXT,
                CVE_ID VARCHAR(255),
                FOREIGN KEY (CVE_ID) REFERENCES CVE_2020_before(CVE_ID)
            )
        """)

        conn.commit()
        conn.close()
        print("Database schema created successfully")
    except mysql.connector.Error as error:
        print("Error while creating database schema:", error)

In [2]:
import mysql.connector
import json
from datetime import datetime

#parsing through .json file and importing data  into database
def parse_and_import_cve_records(file_path):
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="passwordhere",
        database="CVES4"
    )
    cursor = conn.cursor()

    count = 0  #counter variable to import specific num of records

    with open(file_path, 'r') as file:
        for line in file:
            try:
                if count >= 300:
                    break  #loop break

                cve_data = json.loads(line)

                cve_id = cve_data.get("id")
                description = cve_data.get("summary")
                published_date = datetime.strptime(cve_data.get("Published"), "%Y-%m-%d %H:%M:%S")
                last_modified_date = datetime.strptime(cve_data.get("Modified"), "%Y-%m-%d %H:%M:%S")
                CVSS = cve_data.get("cvss")
                cwe = cve_data.get("cwe")
                # Determine the appropriate table based on the CVE publication year
                if published_date.year <= 2020:
                    table_name = "CVE_2020_before"
                else:
                    table_name = f"CVE_{published_date.year}"

                cursor.execute(f"""
                    INSERT IGNORE INTO {table_name} (CVE_ID, Description, Published_Date, Last_Modified_Date, CVSS, CWE)
                    VALUES (%s, %s, %s, %s, %s, %s)
                """, (cve_id, description, published_date, last_modified_date, CVSS, cwe))

                affected_products = cve_data.get("vulnerable_product")
                if affected_products:
                    for product in affected_products:
                        product_info = product.split(':')
                        vendor = product_info[2]
                        product_name = product_info[3]
                        version = product_info[4] if len(product_info) > 4 else None
                        cursor.execute("""
                            INSERT INTO Affected_Products (CVE_ID, Product_Name, Vendor, Version)
                            VALUES (%s, %s, %s, %s)
                        """, (cve_id, product_name, vendor, version))
                
                references = cve_data.get("references")
                if references:
                    for reference in references:
                        # Check if the CVE_ID exists in the appropriate table before inserting into Website table
                        cursor.execute(f"SELECT * FROM {table_name} WHERE CVE_ID = %s", (cve_id,))
                        if cursor.fetchone():  # If the CVE_ID exists, insert into Website table
                            cursor.execute("""
                                INSERT INTO Website (URL, Description, CVE_ID)
                                VALUES (%s, %s, %s)
                            """, (reference, description, cve_id))
                        else:
                            print(f"CVE_ID {cve_id} does not exist in table {table_name}")

                count += 1  #increment for count

            except mysql.connector.Error as error:
                print("Error while parsing and importing CVE records:", error)

    conn.commit()
    conn.close()

In [3]:
# Main function
def main():
    #creating schema
    create_database_schema()

    #file path for .json data
    file_path = "./Documents/CVERecor.json"

    #function call to parse CVE records
    parse_and_import_cve_records(file_path)


if __name__ == "__main__":
    main()

Database schema created successfully
Error while parsing and importing CVE records: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`cves4`.`website`, CONSTRAINT `website_ibfk_1` FOREIGN KEY (`CVE_ID`) REFERENCES `cve_2020_before` (`CVE_ID`))
Error while parsing and importing CVE records: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`cves4`.`website`, CONSTRAINT `website_ibfk_1` FOREIGN KEY (`CVE_ID`) REFERENCES `cve_2020_before` (`CVE_ID`))
Error while parsing and importing CVE records: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`cves4`.`website`, CONSTRAINT `website_ibfk_1` FOREIGN KEY (`CVE_ID`) REFERENCES `cve_2020_before` (`CVE_ID`))
Error while parsing and importing CVE records: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`cves4`.`website`, CONSTRAINT `website_ibfk_1` FOREIGN KEY (`CVE_ID`) REFERENCES `cve_2020_before` (`CVE_ID`))
Error while